In [10]:
# Loading required libraries and fixing the seed
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

SEED = 1
np.random.seed(SEED)

In [11]:
# Loading dataset & preprocessing the data
train_data = pd.read_csv('data/train.csv')
labels_data = pd.read_csv('data/labels.csv')
test_data = pd.read_csv('data/test.csv')

# Removing useless features
X_train = train_data.drop(columns=['Unnamed: 0', 'Age_Group'])
y_train = labels_data['Diabetes_binary']
X_test = test_data.drop(columns=['Unnamed: 0', 'Age_Group'])

# Encoding non-numerical features
def feature_encoding(X):
    non_numerical_columns_names = X.select_dtypes(exclude=['number']).columns
    for column in non_numerical_columns_names:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
    return X

X_train = feature_encoding(X_train)
X_test = feature_encoding(X_test)


# Normalizing features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Implementing functions used for the GridSearch

def custom_f1_threshold(estimator, X, y, threshold=0.5):
    """
    Compute the F1-score with non-default threshold values
    """
    probs = estimator.predict_proba(X)[:, 1]
    preds = (probs >= threshold).astype(int)
    return f1_score(y, preds)


def find_best_params_for_thresholds(model, X_train, y_train, params, thresholds):
    """
    Perform GridSearch with non-default threshold values
    """
    # Implementing Cross validation
    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    
    results = []

    for threshold in thresholds:
        print("\nThreshold:", threshold)
        # grid_search = GridSearchCV(model, param_grid=params, scoring=lambda est, X, y: custom_f1_threshold(est, X, y, threshold), cv=strat_kfold)
        grid_search = GridSearchCV(model, param_grid=params, scoring=lambda est, X, y: custom_f1_threshold(est, X, y, threshold), cv=strat_kfold)
        grid_search.fit(X_train, y_train)

        best_params = grid_search.best_params_
        best_score = grid_search.best_score_

        print("Best parameters:", best_params)
        print("Best F1-Score:", best_score)

        results.append({'threshold': threshold, 'best_params': best_params, 'best_score': best_score})

    return results

In [ ]:
# Performing the GridSearch
param_grid = {
    'C': [0.005, 0.0095, 0.01, 0.02],
    'penalty': ['elasticnet'],
    'solver': ['saga'],
    'l1_ratio': [0.45, 0.5, 0.55],
    'class_weight': ['balanced'],
}

cls = LogisticRegression(random_state=SEED)
threshold = np.linspace(0.595, 0.61, 10)
results = find_best_params_for_thresholds(cls, X_train, y_train, param_grid, threshold)


Threshold: 0.595


In [ ]:
# Extracting the best combinaison
best_f1_score = 0
best_index = 0
for i in range(len(results)):
    best = results[i]['best_score']
    if best > best_f1_score:
        best_f1_score = best
        best_index = i

best_results = results[best_index]
print("\n\n ++++++++++++++++++++++++++++++++++++++++ \n Best Score is", best_f1_score)
print("Threshold is", best_results['threshold'])
print(best_results)

threshold = best_results['threshold']
best_params = best_results['best_params']

In [ ]:
# Training the model with the best hyperparameters
model = LogisticRegression(**best_params, random_state=SEED)
model.fit(X_train, y_train)

# Predict probabilities with the custom threshold
y_train_probs = model.predict_proba(X_train)[:, 1]
y_train_pred = (y_train_probs >= threshold).astype(int)

In [ ]:
# Evaluation on training set
f1 = f1_score(y_train, y_train_pred)
report = classification_report(y_train, y_train_pred, output_dict=True)

print(f"F1-Score on training set for threshold {threshold:.2f}: {f1:.2f}")
print("Classification Report:")
print(classification_report(y_train, y_train_pred))

In [ ]:
# Predicting probabilities on the test set
y_test_probs = model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_probs >= threshold).astype(int)

In [ ]:
# Saving predictions in a .csv file for the Kaggle submission

y_test_pred = pd.DataFrame(y_test_pred, columns=['Diabetes_binary'], index=test_data['Unnamed: 0'])
y_test_pred.index.name = 'index'
y_test_pred.to_csv("test_predictionsLogReg2.csv", index=True)

print(f"Predictions saved to 'test_predictionsLogReg2.csv'")